In [ ]:
# COMMAND ----------
# This notebook defines a Delta Live Tables pipeline focusing on the Raw layer
# following the Raw, Conformed, Curated architecture using batch processing.
# Data is ingested from Unity Catalog Volumes, and partitioning is applied to raw_orders.

import dlt
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, TimestampType, DateType

# Define the base Unity Catalog Volume path where your raw data resides
UC_RAW_INBOUND_PATH = "/Volumes/dbndev/raw/inbound"

# Define specific paths for each entity's raw data within the UC Volume
# For orders, point to the directory containing monthly/daily files
RAW_ORDERS_PATH = f"{UC_RAW_INBOUND_PATH}/orders/"
# For customers and products, point directly to the single JSON file
RAW_CUSTOMERS_PATH = f"{UC_RAW_INBOUND_PATH}/customers/customer_data.json"
RAW_PRODUCTS_PATH = f"{UC_RAW_INBOUND_PATH}/products/product_data.json"

# Define explicit schema for raw orders to prevent _corrupt_record issues with empty/malformed files
# This ensures Spark knows the expected columns, even if data is not present or is corrupt.
orders_explicit_schema = StructType([
    StructField("customer_id", StringType(), True),
    StructField("order_id", StringType(), True),
    StructField("order_date", StringType(), True), # Read as String, then cast later if needed
    StructField("product_id", StringType(), True),
    StructField("quantity", IntegerType(), True),
    StructField("total_price", DoubleType(), True),
    StructField("status", StringType(), True)
    # Add any other expected fields from your raw JSON orders here
])

# Define explicit schema for raw customers
customers_explicit_schema = StructType([
    StructField("customer_id", StringType(), True),
    StructField("first_name", StringType(), True),
    StructField("last_name", StringType(), True),
    StructField("email", StringType(), True),
    StructField("city", StringType(), True),
    StructField("country", StringType(), True),
    StructField("registration_date", StringType(), True) # Read as String, then cast later
])

# Define explicit schema for raw products
products_explicit_schema = StructType([
    StructField("product_id", StringType(), True),
    StructField("product_name", StringType(), True),
    StructField("category", StringType(), True),
    StructField("price", DoubleType(), True),
    StructField("stock_quantity", IntegerType(), True)
])

# COMMAND ----------
# RAW LAYER TABLES (Ingestion from raw sources)
# These tables will reside in the 'raw' schema when deployed (configured via 'target' in DLT settings)

@dlt.table(
    name="raw_orders",
    comment="Raw customer order data ingested from UC Volume (Raw Layer), partitioned by source date.",
    table_properties={"quality": "raw"}
)
def raw_orders():
    """
    Reads raw JSON order data as a batch from the specified UC Volume path.
    Uses an explicit schema to handle potentially empty or malformed files,
    extracts date from filename to use for partitioning (YYYY/MM/DD folder structure).
    - For monthly full loads (e.g., monthly_full_load_2025_01.json), uses YYYY-MM-01.
    - For daily incremental loads (e.g., orders_2025_06_15.json), uses YYYY-MM-DD.
    """
    # Use the explicit schema when reading the JSON files
    df = spark.read.format("json").schema(orders_explicit_schema).load(RAW_ORDERS_PATH)
    
    # Use _metadata.file_path as recommended by Unity Catalog and extract the filename
    df_with_filepath = df.withColumn("source_filepath", F.col("_metadata.file_path"))
    df_with_filename = df_with_filepath.withColumn("actual_filename", F.substring_index(F.col("source_filepath"), '/', -1))

    # Extract date from filename based on patterns
    # Pattern for daily files: orders_YYYY_MM_DD.json
    # Pattern for monthly full load files: monthly_full_load_YYYY_MM.json
    df_with_date = df_with_filename.withColumn(
        "file_date_str",
        F.when(
            F.col("actual_filename").contains("orders_"),
            F.regexp_extract(F.col("actual_filename"), r"orders_(\d{4}_\d{2}_\d{2})\.json", 1)
        ).when(
            F.col("actual_filename").contains("monthly_full_load_"),
            F.regexp_extract(F.col("actual_filename"), r"monthly_full_load_(\d{4}_\d{2})\.json", 1) + "_01" # Add _01 for day
        ).otherwise(
            F.lit("UNKNOWN") # Handle unexpected filenames, or use current date or drop
        )
    )

    # Convert the extracted string date to a proper DateType for partitioning
    # Replace underscores with hyphens for parsing if needed, then cast
    return df_with_date.withColumn(
        "file_date",
        F.to_date(F.regexp_replace(F.col("file_date_str"), "_", "-"), "yyyy-MM-dd").cast(DateType())
    ).drop("source_filepath", "actual_filename", "file_date_str") # Drop intermediate columns


@dlt.table(
    name="raw_customers",
    comment="Raw customer master data ingested from UC Volume (Raw Layer).",
    table_properties={"quality": "raw"}
)
def raw_customers():
    """Reads raw JSON customer data as a batch from the specified UC Volume file."""
    # Use the explicit schema when reading the JSON files
    return spark.read.format("json").schema(customers_explicit_schema).load(RAW_CUSTOMERS_PATH)

@dlt.table(
    name="raw_products",
    comment="Raw product catalog data ingested from UC Volume (Raw Layer).",
    table_properties={"quality": "raw"}
)
def raw_products():
    """Reads raw JSON product data as a batch from the specified UC Volume file."""
    # Use the explicit schema when reading the JSON files
    return spark.read.format("json").schema(products_explicit_schema).load(RAW_PRODUCTS_PATH)


# COMMAND ----------
# CONFORMED LAYER TABLES (Cleaning, Transformation, Conformance)
# These tables are commented out as requested. We will implement SCD Type 2 and other
# transformations here later.

# @dlt.table(
#     name="conformed_dim_customer_product",
#     comment="Cleaned and conformed customer and product dimension data (Conformed Layer).",
#     table_properties={"quality": "conformed", "delta.auto.optimize.optimizeWrite": "true"}
# )
# @dlt.expect_or_drop("valid_customer_id", "customer_id IS NOT NULL")
# @dlt.expect_or_drop("valid_product_id", "product_id IS NOT NULL")
# def conformed_dim_customer_product():
#     customers_df = dlt.read("raw_customers")
#     products_df = dlt.read("raw_products")
#     # Add your join and transformation logic here for customer and product dimensions
#     return (
#         customers_df
#         .join(products_df, customers_df["customer_id"] == products_df["product_id"], "fullouter")
#         .select(
#             F.col("customer_id").alias("dim_customer_id"),
#             F.col("first_name").alias("customer_first_name"),
#             F.col("last_name").alias("customer_last_name"),
#             F.col("email").alias("customer_email"),
#             F.col("city").alias("customer_city"),
#             F.col("country").alias("customer_country"),
#             F.col("registration_date").alias("customer_registration_date"),
#             F.col("product_id").alias("dim_product_id"),
#             F.col("product_name").alias("product_name"),
#             F.col("category").alias("product_category"),
#             F.col("price").alias("product_unit_price"),
#             F.col("stock_quantity").alias("product_stock_quantity")
#         )
#     )

# @dlt.table(
#     name="conformed_fact_sales",
#     comment="Cleaned and validated customer order fact data (Conformed Layer).",
#     table_properties={"quality": "conformed", "delta.auto.optimize.optimizeWrite": "true"}
# )
# @dlt.expect_or_drop("order_id_not_null", "order_id IS NOT NULL")
# @dlt.expect_or_fail("customer_id_not_null", "customer_id IS NOT NULL")
# @dlt.expect("valid_order_date", "order_datetime IS NOT NULL AND order_datetime >= '2020-01-01'")
# @dlt.expect("positive_quantity", "quantity > 0")
# def conformed_fact_sales():
#     orders_df = dlt.read("raw_orders")
#     # conformed_dim_df = dlt.read("conformed_dim_customer_product") # Reads from the conformed dimension
#     cleaned_orders_df = (
#         orders_df
#         .select(
#             F.col("customer_id").cast(StringType()).alias("customer_id"),
#             F.col("order_id").cast(StringType()).alias("order_id"),
#             F.col("order_date").cast(TimestampType()).alias("order_datetime"),
#             F.col("product_id").cast(StringType()).alias("product_id"),
#             F.col("quantity").cast(IntegerType()).alias("quantity"),
#             F.col("total_price").cast(DoubleType()).alias("total_item_price_usd"),
#             F.col("status").cast(StringType()).alias("order_status")
#         )
#     )
#     # Add join with dimension data for enrichment here
#     # return (
#     #     cleaned_orders_df
#     #     .join(conformed_dim_df, ...)
#     #     .select(...)
#     # )
#     return cleaned_orders_df # Returning cleaned_orders_df for now, will be replaced later


# COMMAND ----------
# CURATED LAYER TABLES (Aggregations, Business Views)
# These tables are commented out as requested.

# @dlt.table(
#     name="curated_daily_sales_summary",
#     comment="Aggregated daily sales data (Curated Layer).",
#     table_properties={"quality": "curated"}
# )
# def curated_daily_sales_summary():
#     # return dlt.read("conformed_fact_sales").groupBy(...).agg(...)
#     pass # Placeholder for future implementation

# @dlt.table(
#     name="curated_monthly_customer_spending",
#     comment="Aggregated monthly customer spending (Curated Layer).",
#     table_properties={"quality": "curated"}
# )
# def curated_monthly_customer_spending():
#     # return dlt.read("conformed_fact_sales").groupBy(...).agg(...)
#     pass # Placeholder for future implementation
